In [196]:
import nltk
from nltk.tokenize import TreebankWordTokenizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
from textblob import TextBlob, Word
from nltk.corpus import stopwords
import re
import pandas as pd
from collections import OrderedDict
from operator import getitem
nltk.download('wordnet')
from nltk.corpus import wordnet as wn

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\good5\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [136]:
stop_words = stopwords.words('english')
stop_words.extend(['covid', 'virus', 'https', 'com', 'http', 'que', ' ', 'mierda', 'super', 'new', 'bbmp'])
shortword = re.compile(r'\W*\b\w{1,2}\b')

def lem(a):   
    sent = TextBlob(a)
    result = " ".join([w.lemmatize() for w in sent.words])
    return result   #lemmentization

def process_text(a):
    a = str(re.sub("\S*\d\S*", "", a).strip()) 
    a = a.lower()    
    a = re.sub(r"[^a-zA-Z0-9]"," ",a) #특수문자 제거
    a = re.sub("([^\x00-\x7F])+","",a) #영어이외제거    
    a = re.sub("(.)\\1{3,}", "\\1", a) #긴 반복문 제거
    a = re.sub('https?://\S+|www\.\S+', '', a) #하이퍼링크 제거
    a = shortword.sub(' ', a)    #짧은 단어 제거        
    a = lem(a)    
    tokenizer = TreebankWordTokenizer()
    # tokenize texts
    tokens = tokenizer.tokenize(a)  
    
    result = []
    for b in tokens:
        b = shortword.sub(' ', b)    #짧은 단어 제거  
        if b not in stop_words:  #stopwords 제거
            result.append(b)   
    
    return result

def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

 #명사만 추출

In [137]:
raw_df = pd.read_csv('./data/vaccination_all_tweets.csv')

In [138]:
min(raw_df['date'])

'2020-12-12 11:55:28'

In [139]:
df['process'] = ''
# df_2021.process = df_2021.Content.apply(process_text)
df.process = df.text.apply(process_text)

df['pos'] = df['process'].apply(lambda x: nltk.pos_tag(x)) # token에 pos tagging 

NN_list = [] # 명사 리스트 생성
for i in range(0, len(df['pos'])) :
    NN_words = []
    for word, pos in df['pos'].iloc[i]: 
        if 'NN' in pos: #pos에 nn 이 들어가 있으면 
            NN_words.append(word)
    NN_list.append(NN_words)    

df['nn_list'] = NN_list

In [140]:
min(df['date'])

'2020-12-12 11:55:28'

In [141]:
mask_1 = (df['date'] >= '2020-12-12 00:00:00') & (df['date'] <= '2021-02-11 23:59:59')
df1 = df.loc[mask_1]
df1 = df1.reset_index()

mask_2 = (df['date'] >= '2021-02-12 00:00:00') & (df['date'] <= '2021-05-11 23:59:59')
df2 = df.loc[mask_2]
df2 = df2.reset_index()

mask_3 = (df['date'] >= '2021-05-12 00:00:00') & (df['date'] <= '2021-08-11 23:59:59')
df3 = df.loc[mask_3]
df3 = df3.reset_index()

mask_4 = (df['date'] >= '2021-08-12 00:00:00') & (df['date'] <= '2021-12-31 23:59:59')
df4 = df.loc[mask_4]
df4 = df4.reset_index()

In [193]:
print(len(df1), len(df2), len(df3), len(df4))

7569 62149 94386 64103


In [194]:
for df in [df1, df2, df3, df4]:
    length_arr = []
    nn_arr = []
    for i in range(len(df)):
        length_arr.append(len(df['text'][i]))
        nn_arr.append(len(df['nn_list'][i]))
    print(sum(length_arr)/len(df))
    print(sum(nn_arr)/len(df))
    print("######")

129.85031047694542
5.2973972783723084
######
124.69166036460764
5.33356932533106
######
122.70802873307483
5.665766109380629
######
126.51866527307614
5.970313401868867
######


,index,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet,process,pos,nn_list
0,69718,1395605274950471685,Hürriyet Daily News,Istanbul,Turkey's English news source for more than 60 ...,2009-11-05 08:33:04,279470,75,0,True,2021-05-21 05:00:01,#Turkey seals deal for 60 mln more doses of #P...,"['Turkey', 'PfizerBioNtech', 'vaccine']",TweetDeck,2,2,False,"[turkey, seal, deal, mln, dos, pfizerbiontech,...","[(turkey, JJ), (seal, JJ), (deal, NN), (mln, N...","[deal, mln, dos, pfizerbiontech, vaccine]"
1,69719,1395535037320376321,iLyas BK,"Montréal, Québec",I Build #Startups and Invest In People - Sr So...,2008-07-17 00:32:03,2551,942,2242,False,2021-05-21 00:20:55,Done. #PfizerBiontech https://t.co/9EKcNJXGNr,['PfizerBiontech'],Twitter for iPhone,0,4,False,"[done, pfizerbiontech]","[(done, VBN), (pfizerbiontech, NN)]",[pfizerbiontech]
2,69720,1395511714372784134,kkm,new jersey,real world nurse practitioner--mid-level--term...,2010-09-01 17:46:14,808,297,10160,False,2021-05-20 22:48:14,#Science is wonderful #CovidVaccine #Astrazene...,"['Science', 'CovidVaccine', 'Astrazeneca', 'Pf...",Twitter for iPhone,0,0,False,"[science, wonderful, covidvaccine, astrazeneca...","[(science, NN), (wonderful, JJ), (covidvaccine...","[science, covidvaccine, astrazeneca, pfizerbio..."
3,69721,1395471657603567617,Ms. Nony 💙,East Coast 🇺🇸,"Love to laugh, good conversation, coffee, poli...",2021-01-09 15:17:26,8143,7415,19732,False,2021-05-20 20:09:04,I now have a houseful of half-vaxxed kiddos wi...,['PfizerBiontech'],Twitter Web App,0,16,False,"[houseful, half, vaxxed, kiddos, pfizerbiontec...","[(houseful, JJ), (half, NN), (vaxxed, JJ), (ki...","[half, kiddos, june, household]"
4,69722,1395471023731007491,Ms. Nony 💙,East Coast 🇺🇸,"Love to laugh, good conversation, coffee, poli...",2021-01-09 15:17:26,8143,7415,19732,False,2021-05-20 20:06:33,Fully vaxxed with #PfizerBiontech 🎉 https://t....,['PfizerBiontech'],Twitter Web App,3,154,False,"[fully, vaxxed, pfizerbiontech]","[(fully, RB), (vaxxed, JJ), (pfizerbiontech, NN)]",[pfizerbiontech]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94381,165511,1425268288414576642,VaxBLR,"Bengaluru, India",Hourly updates on FREE and PAID 18+ and 45+ va...,2021-06-21 08:44:34,20,0,0,False,2021-08-11 01:30:14,18-44 #URBAN #Bengaluru #CovidVaccine Availabi...,"['URBAN', 'Bengaluru', 'CovidVaccine', 'COVISH...",VaxBlr,0,0,False,"[urban, bengaluru, covidvaccine, availability,...","[(urban, JJ), (bengaluru, NN), (covidvaccine, ...","[bengaluru, covidvaccine, availability, slot, ..."
94382,165512,1425254676803592193,Julianne Wiley,"Peaceful hills, Upper East TN","Inveterate remedial catechist, Catholic factio...",2020-07-12 15:05:35,512,221,8564,False,2021-08-11 00:36:09,@Shuliicabrera @gregmendozave @telefenoticias ...,['SanLorenzo'],Twitter Web App,0,1,False,"[shuliicabrera, gregmendozave, telefenoticias,...","[(shuliicabrera, NN), (gregmendozave, NN), (te...","[shuliicabrera, gregmendozave, telefenoticias,..."
94383,165513,1425250019154841604,NewsRadio - TNLRN,"Colombo, Sri Lanka","As the News team of TNL Radio Network, we keep...",2010-03-10 10:10:19,4070,11,4,False,2021-08-11 00:17:39,"A stock of 15,000 Russian manufactured Sputnik...",NaN,Twitter for Android,3,12,False,"[stock, russian, manufactured, sputnik, vaccin...","[(stock, NN), (russian, NN), (manufactured, VB...","[stock, russian, vaccine, dos, lanka]"
94384,165514,1425246065729490945,Suriya,NaN,NaN,2011-05-04 08:29:12,13,239,2312,False,2021-08-11 00:01:56,@sputnikvaccine Still why not #WHO approval fo...,"['WHO', 'SputnikV']",Twitter for Android,0,1,False,"[sputnikvaccine, still, approval, sputnikv]","[(sputnikvaccine, NN), (still, RB), (approval,...","[sputnikvaccine, approval, sputnikv]"


## 단어 빈도수 확인

In [143]:
word_list_1 = []
for i in range(len(df1)):
    data = df1['nn_list'][i]
    if len(df1['nn_list'][i])>0:
        word_list_1.extend(data)

word_list_2 = []
for i in range(len(df2)):
    data = df2['nn_list'][i]
    if len(df2['nn_list'][i])>0:
        word_list_2.extend(data)
        
word_list_3= []
for i in range(len(df3)):
    data = df3['nn_list'][i]
    if len(df3['nn_list'][i])>0:
        word_list_3.extend(data)
        
word_list_4 = []
for i in range(len(df4)):
    data = df4['nn_list'][i]
    if len(df4['nn_list'][i])>0:
        word_list_4.extend(data)

## 문장 길이 확인

In [144]:
from collections import Counter
print(Counter(word_list_1).most_common(20))

[('vaccine', 3458), ('pfizerbiontech', 2176), ('pfizer', 945), ('moderna', 487), ('vaccination', 484), ('covidvaccine', 482), ('today', 472), ('coronavirus', 369), ('people', 345), ('day', 328), ('biontech', 253), ('sputnikv', 251), ('amp', 229), ('pfizervaccine', 223), ('health', 221), ('week', 216), ('dos', 215), ('covaxin', 201), ('use', 192), ('dose', 187)]


In [145]:
print(Counter(word_list_2).most_common(20))

[('vaccine', 22625), ('moderna', 11820), ('covaxin', 9355), ('sputnikv', 4757), ('today', 3951), ('pfizer', 3686), ('shot', 3221), ('vaccination', 2992), ('russia', 2557), ('india', 2423), ('day', 2259), ('people', 2114), ('amp', 2022), ('covidvaccine', 1999), ('sinovac', 1959), ('sinopharm', 1839), ('china', 1823), ('case', 1822), ('dos', 1719), ('canada', 1653)]


In [146]:
print(Counter(word_list_3).most_common(20))

[('covaxin', 41483), ('vaccine', 21315), ('slot', 19547), ('age', 17508), ('moderna', 11517), ('hospital', 10651), ('dose', 7243), ('pfizer', 5176), ('sinovac', 4362), ('covishield', 4239), ('vaccination', 4229), ('date', 4161), ('sputnikv', 3999), ('pincode', 3856), ('apollo', 3843), ('fee', 3732), ('today', 3424), ('covidvaccine', 3423), ('india', 3399), ('sinopharm', 3217)]


In [147]:
print(Counter(word_list_4).most_common(20))

[('covaxin', 23522), ('age', 16703), ('slot', 16464), ('vaccine', 14893), ('dose', 10593), ('moderna', 9506), ('pincode', 8489), ('date', 7929), ('fee', 7763), ('covidvaccine', 5968), ('covishield', 5703), ('availability', 5126), ('bengaluru', 5086), ('pfizer', 4770), ('sep', 3818), ('booster', 3289), ('approval', 2965), ('bharatbiotech', 2459), ('health', 2300), ('shot', 2164)]


## 데이터셋에 대한 출력값 차이 비교

In [148]:
!pip install senticnet

In [161]:
from senticnet.senticnet import SenticNet
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from nltk.corpus import sentiwordnet as swn

vader = SentimentIntensityAnalyzer()
sn = SenticNet()

In [176]:
senticnet_dict1 = dict()
vader_dict1 = dict()
for i in range(len(df1)):
    sentence = df1['nn_list'][i]
    for word in sentence:
        try:
            concept_info = sn.concept(word)
            if word in senticnet_dict1:
                senticnet_dict1[word]['count']+=1
            else:
                senticnet_dict1[word] = {'value':concept_info['polarity_value'],'count':1}
        except KeyError as e:
            pass
        try:
            if word in vader.lexicon:
                if word in vader_dict1:
                    vader_dict1[word]['count']+=1
                else:
                    vader_dict1[word] = {'value':vader.lexicon[word], 'count':1}
        except KeyError as e:
            pass

senticnet_dict1 = OrderedDict(sorted(senticnet_dict1.items(), reverse=True,
       key = lambda x: getitem(x[1], 'count')))
vader_dict1 = OrderedDict(sorted(vader_dict1.items(), reverse=True,
       key = lambda x: getitem(x[1], 'count')))

In [181]:
senticnet_dict2 = dict()
vader_dict2 = dict()
for i in range(len(df2)):
    sentence = df2['nn_list'][i]
    for word in sentence:
        try:
            concept_info = sn.concept(word)
            if word in senticnet_dict2:
                senticnet_dict2[word]['count']+=1
            else:
                senticnet_dict2[word] = {'value':concept_info['polarity_value'],'count':1}
        except KeyError as e:
            pass
        try:
            if word in vader.lexicon:
                if word in vader_dict2:
                    vader_dict2[word]['count']+=1
                else:
                    vader_dict2[word] = {'value':vader.lexicon[word], 'count':1}
        except KeyError as e:
            pass
        
senticnet_dict2 = OrderedDict(sorted(senticnet_dict2.items(), reverse=True,
       key = lambda x: getitem(x[1], 'count')))
vader_dict2 = OrderedDict(sorted(vader_dict2.items(), reverse=True,
       key = lambda x: getitem(x[1], 'count')))

In [182]:
senticnet_dict3 = dict()
vader_dict3 = dict()
for i in range(len(df3)):
    sentence = df3['nn_list'][i]
    for word in sentence:
        try:
            concept_info = sn.concept(word)
            if word in senticnet_dict3:
                senticnet_dict3[word]['count']+=1
            else:
                senticnet_dict3[word] = {'value':concept_info['polarity_value'],'count':1}
        except KeyError as e:
            pass
        try:
            if word in vader.lexicon:
                if word in vader_dict3:
                    vader_dict3[word]['count']+=1
                else:
                    vader_dict3[word] = {'value':vader.lexicon[word], 'count':1}
        except KeyError as e:
            pass
        
senticnet_dict3 = OrderedDict(sorted(senticnet_dict3.items(), reverse=True,
       key = lambda x: getitem(x[1], 'count')))
vader_dict3 = OrderedDict(sorted(vader_dict3.items(), reverse=True,
       key = lambda x: getitem(x[1], 'count')))

In [183]:
senticnet_dict4 = dict()
vader_dict4 = dict()
for i in range(len(df4)):
    sentence = df4['nn_list'][i]
    for word in sentence:
        try:
            concept_info = sn.concept(word)
            if word in senticnet_dict4:
                senticnet_dict4[word]['count']+=1
            else:
                senticnet_dict4[word] = {'value':concept_info['polarity_value'],'count':1}
        except KeyError as e:
            pass
        try:
            if word in vader.lexicon:
                if word in vader_dict4:
                    vader_dict4[word]['count']+=1
                else:
                    vader_dict4[word] = {'value':vader.lexicon[word], 'count':1}
        except KeyError as e:
            pass
        
senticnet_dict4 = OrderedDict(sorted(senticnet_dict4.items(), reverse=True,
       key = lambda x: getitem(x[1], 'count')))
vader_dict4 = OrderedDict(sorted(vader_dict4.items(), reverse=True,
       key = lambda x: getitem(x[1], 'count')))

In [192]:
vader_dict4

OrderedDict([('approval', {'value': 2.1, 'count': 2965}),
             ('emergency', {'value': -1.6, 'count': 1466}),
             ('please', {'value': 1.3, 'count': 568}),
             ('miracle', {'value': 2.8, 'count': 465}),
             ('death', {'value': -2.9, 'count': 464}),
             ('thank', {'value': 1.5, 'count': 457}),
             ('risk', {'value': -1.1, 'count': 416}),
             ('thanks', {'value': 1.9, 'count': 379}),
             ('congratulation', {'value': 2.9, 'count': 374}),
             ('share', {'value': 1.2, 'count': 351}),
             ('safety', {'value': 1.8, 'count': 301}),
             ('approves', {'value': 1.7, 'count': 295}),
             ('help', {'value': 1.7, 'count': 292}),
             ('delay', {'value': -1.3, 'count': 285}),
             ('number', {'value': 0.3, 'count': 272}),
             ('care', {'value': 2.2, 'count': 268}),
             ('restriction', {'value': -1.1, 'count': 211}),
             ('flu', {'value': -1.6, 'count': 2

In [204]:
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def penn_to_wn(tag):
    """
    Convert between the PennTreebank tags to simple Wordnet tags
    """
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None

def get_sentiment(word,tag):
    """ returns list of pos neg and objective score. But returns empty list if not present in senti wordnet. """

    wn_tag = penn_to_wn(tag)
    if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
        return []

    lemma = lemmatizer.lemmatize(word, pos=wn_tag)
    if not lemma:
        return []

    synsets = wn.synsets(word, pos=wn_tag)
    if not synsets:
        return []

    # Take the first sense, the most common
    synset = synsets[0]
    swn_synset = swn.senti_synset(synset.name())

    return [swn_synset.pos_score(),swn_synset.neg_score(),swn_synset.obj_score()]

for i in range(len(df4[:1])):
    sentence = df4['nn_list'][i]
    pos_tag = df4['pos'][i]
    senti_val = [get_sentiment(x,y) for (x,y) in pos_tag]
    print(f"pos_val is {pos_tag}")
    print(f"senti_val is {senti_val}")

pos_val is [('one', 'CD'), ('want', 'NN'), ('disgruntled', 'VBD'), ('employee', 'NN'), ('pfizerbiontech', 'NN'), ('right', 'NN')]
senti_val is [[], [0.0, 0.25, 0.75], [], [0.0, 0.0, 1.0], [], [0.0, 0.0, 1.0]]


In [216]:
sentiwordnet_dict1 = dict()
sentiwordnet_dict2 = dict()
sentiwordnet_dict3 = dict()
sentiwordnet_dict4 = dict()

for arr, df in [[sentiwordnet_dict1, df1], [sentiwordnet_dict2, df2], [sentiwordnet_dict3, df3], [sentiwordnet_dict4, df4]]:
    for i in range(len(df)):
        sentence = df['nn_list'][i]
        pos_tag = df['pos'][i]
        senti_val = [get_sentiment(x,y) for (x,y) in pos_tag]
        counter = 0
        for j in senti_val:
            word = pos_tag[counter][0]
            if j:
                if word in arr:
                    arr[word]['count']+=1
                else:
                    arr[word] = {'positive':j[0], 'negative':j[1], 'count':1}

In [210]:
sentiwordnet_dict1 = OrderedDict(sorted(sentiwordnet_dict1.items(), reverse=True,
       key = lambda x: getitem(x[1], 'count')))
sentiwordnet_dict2 = OrderedDict(sorted(sentiwordnet_dict2.items(), reverse=True,
       key = lambda x: getitem(x[1], 'count')))
sentiwordnet_dict3 = OrderedDict(sorted(sentiwordnet_dict3.items(), reverse=True,
       key = lambda x: getitem(x[1], 'count')))
sentiwordnet_dict4 = OrderedDict(sorted(sentiwordnet_dict4.items(), reverse=True,
       key = lambda x: getitem(x[1], 'count')))

In [215]:
sentiwordnet_dict4

OrderedDict([('pincode', {'positive': 0.0, 'negative': 0.0, 'count': 55343}),
             ('urban', {'positive': 0.0, 'negative': 0.0, 'count': 11311}),
             ('moderna', {'positive': 0.75, 'negative': 0.0, 'count': 7947}),
             ('bengaluru',
              {'positive': 0.5, 'negative': 0.125, 'count': 7324}),
             ('covaxin', {'positive': 0.0, 'negative': 0.0, 'count': 6469}),
             ('man', {'positive': 0.0, 'negative': 0.0, 'count': 5277}),
             ('free', {'positive': 0.375, 'negative': 0.0, 'count': 3822}),
             ('got', {'positive': 0.0, 'negative': 0.0, 'count': 3657}),
             ('rural', {'positive': 0.0, 'negative': 0.0, 'count': 3452}),
             ('vaccine', {'positive': 0.0, 'negative': 0.0, 'count': 3270}),
             ('fda', {'positive': 0.0, 'negative': 0.0, 'count': 3183}),
             ('ocgn', {'positive': 0.0, 'negative': 0.0, 'count': 2967}),
             ('pfizer', {'positive': 0.0, 'negative': 0.0, 'count': 2109}),